### **URL 더 똑똑하게 사용하기**

이번에는 템플릿에서 사용한 URL 하드 코딩을 없애는 방법에 대해 알아보자.
URL 하드 코딩이란 무엇일까? question_list.html 템플릿에 사용된 href 값을 보자.

```
<li><a href="/pybo/{{ question.id }}//">{{ question-subject }}</a></li>
```

"/pybo/{{ question.id }}"는 질문 상세를 위한 URL 규칙이다. 하지만 이런 URL 규칙은 프로그램을 수정하면서 '/pybo/question/2/' 또는
'/pybo/2/question/'으로 수정될 가능성도 있다. 이런 식으로 URL 규칙이 자주 변경된다면 모든 href 값들을 일일이 찾아 수정해야 한다.
URL 하드 코딩의 한계인 셈이다.
이런 문제를 해결하려면 해당 URL에 대한 실제 주소가 아닌 주소가 매핑된 URL 별칭을 사용해야 한다.

#### **URL 별칭으로 URL 하드 코딩 문제 해결하기**

* pybo/urls.py 수정하여 URL 별칭 사용하기

템플릿의 href에 실제 주소가 아닌 URL 별칭을 사용하려면 우선 pybo/urls.py 파일을 수정해야 한다.
path 함수에 있는 URL 매핑에 name 속성을 부여한다.

```
from django.urls import path

from . import views


urlpatterns = [
    path('', views.index, name='index'),
    path('<int:question_id>/', views.detail, name='detal'),
]
```

이렇게 수정하면 실제 주소 /pybo/는 index라는 URL 별칭이, /pybo/2/는 detail이라는 URL 별칭이 생긴다.

* pybo/question_list.html 템플릿에서 URL 별칭 사용하기

1단계에서 만든 별칭을 템플릿에서 사용하기 위해 /pybo/{{ question.id }}를 {% url 'detail' question.id %}로 변경한다.
(question.id는 URL 매핑에 정의된 <int:question_id>를 의미한다)

```
(...생략...)
    {% for question in question_list %}
        <li><a href="{% url 'detail' question.id %}">{{ question,subject }}</a></li>
    {% endfor %}
(...생략...)
```

#### **URL 네임스페이스 알아보기**

여기서 한 가지 더 생각할 문제가 있다.
현재 프로젝트에서는 pybo 앱 하나만 사용하지만 이후 pybo 앱 이외의 다른 앱이 프로젝트에 추가될 수도 있다.
이때 서로 다른 앱에서 같은 URL 별칭을 사용하면 중복 문제가 발생한다.

이 문제를 해결하려면 pybo/urls.py 파일에 네임스페이스(namespace)라는 개념을 도입해야 한다.
네임스페이스는 쉽게 말해 각각의 앱이 관리하는 독립된 이름 공간을 의미한다.

* pybo/urls.py에 네임스페이스 추가하기

pybo/urls.py 파일에 네임스페이스를 추가하려면 간단히 app_name 변수에 네임스페이스 이름을 저장하면 된다.

```
from django.urls import path
from . import views

app_name = 'pybo'

urlpatterns = [
    path('', views.index, name='index'),
    path('<int:question_id>/', views.detail, name='detal'),
]
```

네임스페이스 이름으로 'pybo'를 저장했다.

* 네임스페이스 테스트하기 - 오류 발생!

/pybo/에 접속해보자.
하지만 오류가 발생한다.

* pybo/question_list.html 수정하기

오류가 발생한 이유는 템플릿에서 아직 네임스페이스를 사용하고 있지 않기 때문이다.
{% url 'detail' question.id %}을 {% url 'pybo:detail' question.id %}로 바꾸자.

```
{% for question in question_list %}
    <li><a href="/pybo/{{ question.id }}/">{{ question.subject }}</a></li>
    <li><a href="{% url 'pybo:detail' question.id %}">{{ question.subject }}</a></li>
{% endfor %}
```

detial에 pybo라는 네임스페이스를 붙여준 것이다.

### **답변 등록 기능 만들기**

앞에서 질문 등록, 조회 기능을 만들었다.
이번에는 답변 등록과 답변을 보여주는 기능을 만들어 보자.

#### 답변 저장하고 표시하기

* 질문 상세 템플릿에 답변 등록 버튼 만들기

질문 상세 템플릿 pybo/question_detail.html 파일을 수정한다.
form 엘리먼트 안에 textarea 엘리먼트와 input 엘리먼트를 포함시켜 답변 내용, 답변 등록 버튼을 추가한다.

```
<h1>{{ question.subject }}</h1>

<div>
    {{ question.content }}
</div>

<form action="{% url 'pybo:answer_create' question.id} %" method="post">
    {% csrf_token %}
    <textarea name="content" id="content" rows="15"></textarea>
    <input type="submit" value="답변 등록">
</form>
```

<답변 등록> 버튼을 누를 때 호출되는 URL은 action 속성에 있는 {% url 'pybo:answer_create' question.id %}이다.
그리고 form 엘리먼트 바로 아래에 있는 {% csrf_token %}이 눈에 띈다.
이 코드는 보안 관련 항목이니 잠시 뒤에 설명하겠다.
{% csrf_token %}는 form 엘리먼트를 통해 전송된 데이터(답변)가 실제로 웹 브라우저에서 작성된 데이터인지 판단한다.
그러므로 <form ...> 태그 바로 밑에 {% csrf_token %}을 항상 입력해야 한다. 
해킹처럼 올바르지 않은 방법으로 데이터가 전송되면 서버에서 발행한 csrf_token 값과 해커가 보낸 csrf_token 값이 일치하지 않으므로 오류를 발생시켜
보안을 유지할 수 있다.

* 답변 등록을 위한 URL 매핑 등록하기

pybo/urls.py 파일에 답변 등록을 위한 URL 매핑을 등록한다.

```
urlpatterns = [
    path('', views.index, name='index'),
    path('<int:question_id>/', views.detail, name='detal'),
    path('answer/create/<int:question_id>/', views.answer_create, name='answer_create'),
]
```

이 코드는 사용자가 상세 화면에서 <질문 답변> 버튼을 눌렀을 때 작동할 form 엘리먼트의 pybo/answer/create/2/에 대한 
URL 매핑을 추가한 것이다.

* answer_create 함수 추가하기

form 엘리먼트에 입력된 값을 받아 데이터베이스에 저장할 수 있도록 answer_create 함수를 pybo/views.py 파일에 추가한다.

```
from django.shortcuts import (
    render,
    get_object_or_404,
    redirect,
)
from .models import Question
from django.utils import timezone


def answer_create(request, question_id):
    """
    pybo 답변 등록
    """
    
    question = get_object_or_404(Question, pk=question_id)
    question.answer_set.create(
        content = request.POST.get('content'),
        create_date = timezone.now(),
    )
```

answer_create 함수의 question_id 매개변수에는 URL 매핑 정보값이 넘어온다.
예컨대 /pybo/answer/create/2가 요청되면 question_id에는 2가 넘어온다.
request 매개변수에는 pybo/question_detial.html에서 textarea에 입력된 데이터가 파이썬 객체에 담겨 넘어온다.
이 값을 추출하기 위한 코드가 바로 request.POST.get('content')이다.

그리고 Question 모델을 통해 Answer 모델 데이터를 생성하기 위해 question.answer_set.create를 사용했다.

```
* Answer 모델을 통해 데이터를 저장할 수도 있다.

위에서는 Answer 모델 데이터 저장을 위해 Question 모델을 사용했지만,
Answer 모델을 직접 사용해도 저장할 수 있다.

    # answer = Answer(question=question, content=request.POST.get('content'), create_date=timezone.now())
    # answer.save()
```

* 답변 등록 후 상세 화면으로 이동하게 만들기

